<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Fine-tuning-a-deep-learning-model-with-HuggingFace-🤗Datasets-library-and-PyTorch" data-toc-modified-id="Fine-tuning-a-deep-learning-model-with-HuggingFace-🤗Datasets-library-and-PyTorch-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Fine-tuning a deep-learning model with HuggingFace <code>🤗Datasets</code> library and PyTorch</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Load-the-model-and-the-tokenizer" data-toc-modified-id="Load-the-model-and-the-tokenizer-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Load the model and the tokenizer</a></span></li><li><span><a href="#Load-and-process-the-dataset" data-toc-modified-id="Load-and-process-the-dataset-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Load and process the dataset</a></span><ul class="toc-item"><li><span><a href="#Tokenizing-the-dataset" data-toc-modified-id="Tokenizing-the-dataset-1.0.2.1"><span class="toc-item-num">1.0.2.1&nbsp;&nbsp;</span>Tokenizing the dataset</a></span></li><li><span><a href="#Formatting-the-dataset" data-toc-modified-id="Formatting-the-dataset-1.0.2.2"><span class="toc-item-num">1.0.2.2&nbsp;&nbsp;</span>Formatting the dataset</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-1.0.2.3"><span class="toc-item-num">1.0.2.3&nbsp;&nbsp;</span>Training</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Fine-tuning a deep-learning model with HuggingFace `🤗Datasets` library and PyTorch

Based on its [Quick tour example](https://huggingface.co/docs/datasets/quicktour.html)

Colab author: Manuel Romero / [@mrm8488](https://twitter.com/mrm8488)

In [3]:
# Make sure that we have a recent version of pyarrow in the session before we continue - otherwise reboot Colab to activate it
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

### Load the model and the tokenizer

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

### Load and process the dataset

In [5]:
from datasets import load_dataset
dataset = load_dataset('glue', 'mrpc', split='train')

Reusing dataset glue (/home/quan/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [7]:
len(dataset)

3668

In [9]:
dataset[0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

In [10]:
dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
# dataset.filter(lambda example: example['label'] == dataset.features['label'].str2int('equivalent'))[0]

In [ ]:
# dataset.filter(lambda example: example['label'] == dataset.features['label'].str2int('not_equivalent'))[0]

#### Tokenizing the dataset

In [11]:
def encode(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')
dataset = dataset.map(encode, batched=True)


Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-ecb3eae944990a41.arrow


In [12]:
list(dataset[0].keys())

['attention_mask',
 'idx',
 'input_ids',
 'label',
 'sentence1',
 'sentence2',
 'token_type_ids']

In [26]:
len(dataset[0]['input_ids']),len(dataset[0]['attention_mask']),len(dataset[0]['token_type_ids'])

(512, 512, 512)

In [34]:
print(dataset[0]['input_ids'])
print(dataset[0]['input_ids'].index(0))

[101, 7277, 2180, 5303, 4806, 1117, 1711, 117, 2292, 1119, 1270, 107, 1103, 7737, 107, 117, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102, 11336, 6732, 3384, 1106, 1140, 1112, 1178, 107, 1103, 7737, 107, 117, 7277, 2180, 5303, 4806, 1117, 1711, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [32]:
print([tokenizer.decode(i) for i in dataset[0]['input_ids']])

['[CLS]', 'Am', '##ro', '##zi', 'accused', 'his', 'brother', ',', 'whom', 'he', 'called', '"', 'the', 'witness', '"', ',', 'of', 'deliberately', 'di', '##sto', '##rting', 'his', 'evidence', '.', '[SEP]', 'Re', '##fer', '##ring', 'to', 'him', 'as', 'only', '"', 'the', 'witness', '"', ',', 'Am', '##ro', '##zi', 'accused', 'his', 'brother', 'of', 'deliberately', 'di', '##sto', '##rting', 'his', 'evidence', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

In [38]:
print(dataset[0]['token_type_ids']) 
print(dataset[0]['token_type_ids'].index(1)) # to signal the 2nd sentence

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [35]:
print(dataset[0]['attention_mask'])
print(dataset[0]['attention_mask'].index(0))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [21]:
print(dataset[0]['sentence1'])

Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .


In [22]:
print(dataset[0]['sentence2'])

Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .


512


#### Formatting the dataset

In [42]:
dataset = dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

In [43]:
dataset[0]['label'],dataset[0]['labels']

(1, 1)

In [44]:
import torch
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)
next(iter(dataloader))

/home/quan/anaconda3/envs/fastai_v2/lib/python3.8/site-packages/datasets/arrow_dataset.py:850: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[  101,  7277,  2180,  ...,     0,     0,     0],
         [  101, 10684,  2599,  ...,     0,     0,     0],
         [  101,  1220,  1125,  ...,     0,     0,     0],
         ...,
         [  101, 16944,  1107,  ...,     0,     0,     0],
         [  101,  1109, 11896,  ...,     0,     0,     0],
         [  101,  1109,  4173,  ...,     0,     0,     0]]),
 'labels': tensor([1, 0, 1, 0, 1, 1, 0, 1]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]])}

In [47]:
tmp = next(iter(dataloader))

In [50]:
len(tmp) # 4 keys

4

In [51]:
tmp['attention_mask'].shape # bs = 8

torch.Size([8, 512])

#### Training

In [45]:
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [46]:
model.train().to(device)

# note that we let Adam optimizer to access all the parameters in the model for Adam GD
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

In [52]:
type(model)

transformers.models.bert.modeling_bert.BertForSequenceClassification

In [55]:
tmp_cuda =  {k: v.to(device) for k, v in tmp.items()}

tmp_outp = model(**tmp_cuda)

In [56]:
len(tmp_outp) # output of huggingface model is always a tuple.


2

In [59]:
# for this case, the first element is the loss (negative log loss, as we want to see if 2 sentences are equivalent)
# (since the model we are using is 'SequenceClassification?')
tmp_outp[0]

tensor(0.7886, device='cuda:0', grad_fn=<NllLossBackward>)

In [63]:
tmp_outp[1].shape

torch.Size([8, 2])

In [64]:
tmp_outp[1] # the 2nd element is just the head output (presoftmax)

tensor([[ 0.3832,  0.0596],
        [ 0.2036, -0.1933],
        [ 0.1324, -0.3716],
        [ 0.4472,  0.1715],
        [ 0.2803, -0.1362],
        [ 0.4707,  0.0265],
        [ 0.4794, -0.0466],
        [ 0.3093, -0.3229]], device='cuda:0', grad_fn=<AddmmBackward>)

In [65]:
for epoch in range(3):
    for i, batch in enumerate(tqdm(dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

  0%|          | 2/459 [00:00<01:28,  5.17it/s]

loss: 0.7385174632072449


  3%|▎         | 12/459 [00:01<01:09,  6.41it/s]

loss: 0.6396863460540771


  5%|▍         | 22/459 [00:03<01:08,  6.36it/s]

...

 96%|█████████▋| 442/459 [01:12<00:02,  6.14it/s]

loss: 0.4335087537765503


 98%|█████████▊| 452/459 [01:14<00:01,  6.04it/s]

loss: 0.10622315853834152


100%|██████████| 459/459 [01:15<00:00,  6.08it/s]
